In [1]:
import requests
import pandas as pd
from pandas.core.common import flatten
import numpy as np
import re
import random
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
# pressing certain keys
# from selenium.webdriver.common.keys import Keys
import time
import pickle
from datetime import datetime
# using beautiful soup to find our data a bit easier
# from bs4 import BeautifulSoup
# also trying gazpacho soup
from gazpacho import Soup
# convert html to markdown
from markdownify import markdownify as md
# for US states
import us

In [26]:
# login via cookie
BASE_URL = 'https://www.linkedin.com'
def login_with_cookie(cookie_path):
    # first load chromedriver
    OPTIONS = webdriver.ChromeOptions()
    # make the browser pretty thin so the page doesn't have multiple scroll bars
    # width,height
    OPTIONS.add_argument("window-size=600,800")
    OPTIONS.headless = True
    # remove all these when watching it the first time
    # if we need to be a realer browser (not a headless chrome)
    print('opening chrome')
    driver = webdriver.Chrome('chromedriver',options=OPTIONS)
    # login with cookie, how long is cookie timeout? unknown
    # think about logging in via username and pass
    COOKIE_NAME = 'linkedin_cookies_2021-02-14.pkl'
    # load the page without the cookie
    driver.get(BASE_URL)
    # load the cookie
    with open(cookie_path, "rb") as f:
        cookies = pickle.load(f)  
    # cookies = pickle.load(open('meetup_cookies.pkl','rb'))    
    print('loading cookies')
    for cookie in cookies:
        driver.add_cookie(cookie)
    # load the page again after the cookie is loaded, this is the same as logging in
    print('opening chrome with cookies to login')
    driver.get(BASE_URL)
    # minimise the messages area
    minimise_msgs(driver)
    return driver

In [3]:
# minimise the messaging bit if it's open
def minimise_msgs(driver):
    # if this is open
    msg_min = len(driver.find_elements_by_class_name('msg-overlay-list-bubble--is-minimized'))
    if msg_min == 0:
        try:
            print('minimising the messages overlay')
            msg_buttons = driver.find_elements_by_class_name('msg-overlay-bubble-header__control')
            msg_buttons[1].click()
        except:
            pass
    else:
        pass

In [4]:
# we scroll in chunks because things don't load if we scroll to the bottom immediately
def scroll_to_bottom2(driver,scroll_num):
    pause = 2 # one sec is def too fast, script starts to fail to find the next page button
    while(scroll_num>0):
        print('scrolling down 800 pixels')
        driver.execute_script("window.scrollBy(0,800);")
        time.sleep(pause)
        scroll_num -= 1

In [5]:
# safest to get the loc manually via the URL
# e.g. location=Sydney%2C%20New%20South%20Wales%2C%20Australia
# first run basically finds the pagination -- fix the pagination.. look at the last number and make that the last page
# next loop grabs all job links
def search_grab_links_v3(driver,kws,loc,top_pages='1'):
    kws = kws.replace(' ','%20').replace(',','%2C')
    URL = BASE_URL + '/jobs/search/?keywords='+kws+'&'+'location='+loc
    # get the search engine results page (SERP)
    driver.get(URL)
    # add a sleep time, so the page is fully loaded before we scroll
#     time.sleep(3)
    # find the minimise messaging button, there's only 2 with this class
    # the first is the write new message button
    # only need to do this once because we're in the same session
    minimise_msgs(driver)
    # scroll to the bottom to load all the jobs, linkedin takes about 5x scrolls of 800px
    scroll_to_bottom2(driver,scroll_num=5)
    # grab the page's html as a string
    html = driver.page_source
    soup = Soup(html)
    # find total pages we have
    # driver.find_elements_by_xpath('//button[contains(@aria-label,"Page")]')[-1].text
    pagi = soup.find('li', {'class': 'pagination'}, partial=True)[-1].find('span').text
    # find all the job links anchor tags
    job_links1 = soup.find('a', {'class':'job-card-list__title'})
    # find all href links in the a tags, and cut them to minimum size /jobs/view/123456789/
    try:
        job_links = [re.search('^(.+)\?',x.attrs['href']).group(1) for x in job_links1]
    except Exception as e:
        job_links = ['nolink']
        print(f'Exception is {e}')
    # put the job links in our all_job_links list
    all_job_links = []
    all_job_links.append(job_links)
    
    # there's a chance there could be say 20 pages, and we don't want to wait this long
    if(top_pages == '1'):
        return all_job_links
    # pages is always a str for both below scenarios
    elif(top_pages=='all'):
        pages = pagi
    else:
        pages = top_pages
    
    # loop covers grabbing job links
    for i in range(2,int(pages)+1):
        # click the next page button, will click 2, since we're on 1 first
        try:
            print(f'clicking page button {i}')
#             driver.find_element_by_css_selector(f"button[aria-label='Page {i}']").click()
            driver.find_element_by_xpath(f"//button/span[contains(text(), '{i}')]/ancestor::button").click()
        except Exception as e:
            # If pop-up overlay appears, click the X button to close
            time.sleep(2) # Sometimes the pop-up takes time to load
            print(f'error {e}, retrying clicking page button {i} with …')
            driver.find_element_by_xpath(f"//button/span[contains(text(), '…')]/ancestor::button").click()
#             driver.find_element_by_css_selector(f"button[aria-label='Page {i}']").click()
        # wait for site to load
        time.sleep(1)
        # scroll to bottom
        scroll_to_bottom2(driver,scroll_num=5)
        # grab the page source
        html = driver.page_source
        # make it a soup
        soup = Soup(html)
        # find total pages we have
#         pagi = soup.find('li', {'class': 'pagination'}, partial=True)[-1].find('span').text
        # find all the job links anchor tags
        job_links1 = soup.find('a', {'class':'job-card-list__title'})
        try:
            job_links = [re.search('^(.+)\?',x.attrs['href']).group(1) for x in job_links1]
        except Exception as e:
            job_links = ['nolink']
            print(f'Exception is {e}')
        # put the job links in our all_job_links list
        all_job_links.append(job_links)
    
    # all_job_links is a list of lists, this last bit flattens it into a single list
    return list(flatten(all_job_links))

In [6]:
# safest to get the loc manually via the URL
# e.g. location=Sydney%2C%20New%20South%20Wales%2C%20Australia
# first run basically finds the pagination -- fix the pagination.. look at the last number and make that the last page
# next loop grabs all job links
def search_grab_links_v2(driver,kws,loc,top_pages='1'):
    kws = kws.replace(' ','%20').replace(',','%2C')
    URL = BASE_URL + '/jobs/search/?keywords='+kws+'&'+'location='+loc
    # get the search engine results page (SERP)
    driver.get(URL)
    # add a sleep time, so the page is fully loaded before we scroll
#     time.sleep(3)
    # find the minimise messaging button, there's only 2 with this class
    # the first is the write new message button
    # only need to do this once because we're in the same session
    minimise_msgs(driver)
    # scroll to the bottom to load all the jobs, linkedin takes about 5x scrolls of 800px
    scroll_to_bottom2(driver,scroll_num=5)
    # grab the page's html as a string
    html = driver.page_source
    soup = Soup(html)
    # find total pages we have
    # driver.find_elements_by_xpath('//button[contains(@aria-label,"Page")]')[-1].text
    pagi = soup.find('li', {'class': 'pagination'}, partial=True)[-1].find('span').text
    # find all the job links anchor tags
    job_links1 = soup.find('a', {'class':'job-card-list__title'})
    # find all href links in the a tags, and cut them to minimum size /jobs/view/123456789/
    try:
        job_links = [re.search('^(.+)\?',x.attrs['href']).group(1) for x in job_links1]
    except Exception as e:
        job_links = ['nolink']
        print(f'Exception is {e}')
    # put the job links in our all_job_links list
    all_job_links = []
    all_job_links.append(job_links)
    
    # there's a chance there could be say 20 pages, and we don't want to wait this long
    if(top_pages == '1'):
        return all_job_links
    # pages is always a str for both below scenarios
    elif(top_pages=='all'):
        pages = pagi
    else:
        pages = top_pages
    
    # loop covers grabbing job links
    for i in range(2,int(pages)+1):
        # click the next page button, will click 2, since we're on 1 first
        try:
            print(f'clicking page button {i}')
            driver.find_element_by_css_selector(f"button[aria-label='Page {i}']").click()
        except Exception as e:
            # If pop-up overlay appears, click the X button to close
            time.sleep(2) # Sometimes the pop-up takes time to load
            print(f'error {e}, retrying clicking page button {i}')
            driver.find_element_by_css_selector(f"button[aria-label='Page {i}']").click()
        # wait for site to load
        time.sleep(1)
        # scroll to bottom
        scroll_to_bottom2(driver,scroll_num=5)
        # grab the page source
        html = driver.page_source
        # make it a soup
        soup = Soup(html)
        # find total pages we have
#         pagi = soup.find('li', {'class': 'pagination'}, partial=True)[-1].find('span').text
        # find all the job links anchor tags
        job_links1 = soup.find('a', {'class':'job-card-list__title'})
        try:
            job_links = [re.search('^(.+)\?',x.attrs['href']).group(1) for x in job_links1]
        except Exception as e:
            job_links = ['nolink']
            print(f'Exception is {e}')
            
        # put the job links in our all_job_links list
        all_job_links.append(job_links)
    
    # all_job_links is a list of lists, this last bit flattens it into a single list
    return list(flatten(all_job_links))

In [7]:
# use some dirty method of cleaning this html so we can run NLP on it later
def soup_to_md(soup):
    # remove the divs that surround, gazpacho doesn't seem to have a way to select all
    temp1 = re.search('\n(.+)\n',str(soup)).group(1)
    # make it markdown to remove other tags
    temp2 = md(temp1)
    # remove \n created by markdown
    temp3 = temp2.replace('\n','')
    # remove *'s created by markdown
    temp4 = temp3.replace('*','')
    # replace >1 space with just 1 space
    temp5 = re.sub(' +',' ',temp4)
    return temp5

In [8]:
# 5 sec x 100 = 500 sec ≈ 8.3min
# 5 sec x 150 = 750 sec ≈ 12.5min
# 5 sec x 200 = 1000 sec ≈ 16.6min
# make the script to follow each of these links and grab the nice text
def grab_job_descs_v2(links):
    total_links = len(links)
    out = []
    
    for i,link in enumerate(links):
        try:
            print(f'grabbing link {link}, {i+1} of {total_links}')
            out.append(grab_desc_v2(link))
            time.sleep(5)
        except:
            print(f'this link failed {link}')
            
    return out

In [9]:
# clean this job details bit
def grab_job_details(soup):
    job_extra_keys_raw = soup.find('li', {'class': 'job-criteria'}, partial=True, mode='all')
    job_extra_vals_raw = soup.find('span', {'class': 'job-criteria'}, partial=True, mode='all')
    
    # some logic based on number of list items found
    num_keys = len(job_extra_keys_raw)
    if num_keys == 0 :
        return {}
    if num_keys == 1 :
        key = job_extra_keys_raw[0].text.title()
        val = job_extra_vals_raw[0].text
        return {key:val}
    else :
        job_extra_keys = [key.text.title() for key in job_extra_keys_raw]
        job_extra_vals = [val.text for val in job_extra_vals_raw]
        return dict(zip(job_extra_keys,job_extra_vals))

In [10]:
# what we do here is odd, this request is without login
# this is to preserve some kind of purity in your logged in account
# one consequence of this is that some jobs might not be public
# 2nd is that css and html is actually different for logged in vs public
def grab_desc_v2(link):
    BASE_URL = 'https://www.linkedin.com'
    URL = BASE_URL + link 
    res = requests.get(url=URL)
    # using this gazpacho branded Soup
    soup = Soup(res.text)
    # they all live in these *topcard* classes
    # job_card contains the organisation and location
    # job_title lives above the card in the h1
    job_card = soup.find('span', {'class': 'topcard'}, partial=True)
    job_title = soup.find('h1', {'class': 'topcard'}, partial=True).text
    job_org = job_card[0].text
    job_loc = job_card[1].text
    # grab job description
    soup_desc = soup.find('div', {'class': 'show-more-less'}, partial=True)
    job_desc = soup_to_md(soup_desc)
    # grab job extra details
    # there is a variable amount of these, sometimes there might be none
    # so we offshore our cleaning
    job_extra_details = grab_job_details(soup)
    return {
        'Job Title':job_title,
        'Company':job_org,
        'Job Location':job_loc,
        'Job URL':URL,
        'Job Description':job_desc,
        'Job Details':job_extra_details
    }

In [11]:
# job list to df
def joblist_to_df(listt):
    # make the df
    df1 = pd.DataFrame(listt)
    # json normalize the extra details
    df2 = pd.json_normalize(df1['Job Details'])
    # append extra details
    df3 = pd.concat([df1,df2],axis='columns')
    # remove the old extra details column
    df = df3.drop(columns=['Job Details'])
    return df

In [27]:
%%time
# making the first good data frame
# login to linkedin with cookie
driver = login_with_cookie(cookie_path='linkedin_cookies_2021-02-14.pkl')
# use selenium to copy all job links
# https://www.linkedin.com/jobs/search/?f_PP=104769905%2C100992797&geoId=101452733&keywords=machine%20learning%20engineer&location=Australia&sortBy=R
keywords = 'solution engineer'
location = 'Australia'
all_job_links = search_grab_links_v3(driver=driver,kws=keywords,loc=location,top_pages='all')
# use requests and gazpacho to get all the details of a job
jobs = grab_job_descs_v2(all_job_links)
# make the list into a dataframe
jobs_df = joblist_to_df(jobs)
# drop the duplicates -- linkedin displays the same job from different recruiters sometimes
jobs_df = jobs_df.drop_duplicates(subset=['Job Title', 'Company', 'Job Location', 'Job Description',
       'Seniority Level', 'Employment Type', 'Job Function', 'Industries'])
jobs_df = jobs_df.reset_index(drop=True)
jobs_df

opening chrome
loading cookies
opening chrome with cookies to login
minimising the messages overlay
minimising the messages overlay
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
clicking page button 2
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
clicking page button 3
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
clicking page button 4
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
clicking page button 5
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
clicking page button 6
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels
scrolling down 800 pixels

,Job Title,Company,Job Location,Job URL,Job Description,Seniority Level,Employment Type,Job Function,Industries
0,Associate Solution Engineer - Sydney,Appian Corporation,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2622582377/,"At Appian, we’re seeking to grow our culture ...",Associate,Full-time,Engineering and Information Technology,Computer Software
1,Presales solution Engineer,Powell Software,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2649187443/,,Associate,Full-time,Sales and Information Technology,Information Technology and Services
2,Service Cloud Solution Engineer,Salesforce,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/2652778606/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ..."
3,Associate Solution Engineer,Salesforce,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2661212338/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ..."
4,Solution Engineer,Salesforce,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2661268390/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ..."
...,...,...,...,...,...,...,...,...,...
561,RAMS Engineer,Jobs on jobactive,"Docklands, Victoria, Australia",https://www.linkedin.com/jobs/view/2659088142/,We currently have a career defining opportuni...,Entry level,Full-time,Engineering and Information Technology,"Restaurants, Food & Beverages, and Design"
562,Field Services Engineer - Mechanical (Turbomac...,Baker Hughes,"Moomba, South Australia, Australia",https://www.linkedin.com/jobs/view/2588177238/,"This is not an active position, this posting ...",Entry level,Full-time,Information Technology,"Information Technology and Services, Computer ..."
563,Mechanical Engineer,Wave International Pty Ltd,"Brisbane, Queensland, Australia",https://www.linkedin.com/jobs/view/2650782889/,Leading Engineering Consultancy working from ...,Entry level,Full-time,Engineering and Information Technology,"Staffing and Recruiting, Oil & Energy, and Man..."
564,Dynamics 365 F&O Solution Architect,Microsoft,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/2486220651/,The Microsoft Asia Business Application Domai...,Not Applicable,Full-time,Engineering and Information Technology,"Computer Hardware, Computer Software, and Info..."


In [63]:
# the li[number] will do for now for a quick fix
# driver.find_element_by_xpath('/html/body/div[5]/div[3]/div[3]/div/div/section[1]/div/div/section/div/ul/li[9]/button').click()


In [24]:
%%time
jobs = grab_job_descs_v2(all_job_links)
# make the list into a dataframe
jobs_df = joblist_to_df(jobs)
# drop the duplicates -- linkedin displays the same job from different recruiters sometimes
jobs_df = jobs_df.drop_duplicates(subset=['Job Title', 'Company', 'Job Location', 'Job Description',
       'Seniority Level', 'Employment Type', 'Job Function', 'Industries'])
jobs_df = jobs_df.reset_index(drop=True)
jobs_df

NameError: name 'all_job_links' is not defined

# Now we have our dataframe

In [92]:
jobs_df

,Job Title,Company,Job Location,Job URL,Job Description,Seniority Level,Employment Type,Job Function,Industries
0,Data & Quality Manager,Metcash,"Macquarie Park, New South Wales, Australia",https://www.linkedin.com/jobs/view/2658781767/,Join an award winning team who have created o...,Mid-Senior level,Full-time,Information Technology,"Food & Beverages, Consumer Goods, and Retail"
1,Senior Manager Digital Insights Analytics,Commonwealth Bank,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2651938665/,Senior ManagerData and Decision Science(12 mo...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology","Information Technology and Services, Banking, ..."
2,Analytics and Insights Manager,Domain,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2615561143/,Analytics and Insights Manager - Sydney - Per...,Mid-Senior level,Full-time,"Analyst, Finance, and Research","Information Technology and Services, Online Me..."
3,Data and Analytics Manager,Macquarie Group,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2645644230/,Data and Analytics ManagerSydney Are you a Da...,Not Applicable,Full-time,Finance,"Banking, Financial Services, and Investment Ba..."
4,Analytics & Insights Manager,Nine,"North Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2650968303/,Department/Team OverviewThe Product team is r...,Mid-Senior level,Full-time,Information Technology,Online Media
...,...,...,...,...,...,...,...,...,...
479,Marketing Analytics Manager,Sbs,"New South Wales, Australia",https://www.linkedin.com/jobs/view/2644918260/,There’s a reason people work at SBS. SBS is o...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology","Marketing and Advertising, Online Media, and B..."
480,EPMO Portfolio Financial & Reporting Manager,Optus,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2608215470/,Optus’ vision is to be Australia’s most loved...,Mid-Senior level,Full-time,Accounting/Auditing and Finance,"Telecommunications, Internet, and Information ..."
481,Analyst - Quantitative Modelling,Bank of Queensland,"Newstead, Queensland, Australia",https://www.linkedin.com/jobs/view/2520437408/,Join an ASX 100 finance leader! Exciting grow...,Not Applicable,Full-time,Business Development and Sales,Banking
482,Data Scientist,ANZ,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/2645676830/,Data Scientist - Customer Resolutions Locatio...,Not Applicable,Full-time,Engineering and Information Technology,"Accounting, Banking, and Financial Services"


# Creating the final tables begins

## Feature Engineering for our Raw Data - Adding Salary and fixing some locations

In [28]:

filter_dollars = jobs_df['Job Description'].str.contains('\$')
# the below styles should be tailored for your country
# salary style1 $100,000 or $123456
reg_salary1 = '\$\d{2,3},{0,1}\d{3}'
filter_salary1 = jobs_df['Job Description'].str.contains(reg_salary1)
# salaray style2 = $100k or $123 k
reg_salary2 = '\$\d{2,3} {0,1}k'
filter_salary2 = jobs_df['Job Description'].str.contains(reg_salary2)
# jobs_df[filter_dollars]['Job Description'].str.extract('(\$\d{2,3},{0,1}\d{3})')
# jobs_df['Job Description'].str.extract('(\$\d{2,3} {0,1}k)')
# jobs_df[filter_dollars]['Job Description'].iloc[6]

In [29]:
# based on the patterns that salaries are posted in
# should be tailored for your country
# salary style1 $100,000 or $123456
# salaray style2 = $100k or $123 k
def grab_salary(jobs_df,salary_patterns):
    # the broadest filter, if they have a $ sign, should change this symbol for your own country
    # alternatively the broadest filter can be the word "salary" or "wage", etc
    filter_dollars = jobs_df['Job Description'].str.contains('\$')
    temp_output = []
    for pattern in salary_patterns:
        # setup the filter first
        filter_salary = jobs_df['Job Description'].str.contains(pattern)
        # create the df and extract all from that pattern
        temp_df = jobs_df[filter_salary]['Job Description'].str.extractall(f'({pattern})')
        # unstack puts all results in a new col rather but is still multiindex
        temp_df = temp_df.unstack()
        # this makes all salaries into style1
        temp_df = temp_df.apply(lambda x: x.str.replace('k',',000'))
        # remove the redundant multiindex
        temp_df.columns = temp_df.columns.droplevel()
        # add it to our list
        temp_output.append(temp_df)
    
    # return the concat of our lists
    return pd.concat(temp_output,axis='columns')

In [30]:
# do some cleaning on our salaries
# mainly because companies sometimes put a single salary 
# and sometimes put a salary range
# we want to compare average salaries between companies
def fix_salary(salary_df):
    # remove duplicates if a company somehow put both salary forms
    salary_df = salary_df.reset_index().drop_duplicates(subset='index').set_index('index')
    # remove the names of the columns and index
    salary_df.index.name=None
    salary_df.columns.name=None
    # make a dataframe of numbers so we can do math on it
    salary_df_num = salary_df.apply(lambda x: pd.to_numeric(x.str.replace('\$|,',''),errors='ignore',downcast='integer'))
    # min, max and avg salary in numbers
    salary_df['Min_Salary_Num'] = salary_df_num.min(axis='columns')
    salary_df['Max_Salary_Num'] = salary_df_num.max(axis='columns')
    salary_df['Avg_Salary_Num'] = salary_df[['Min_Salary_Num','Max_Salary_Num']].mean(axis='columns')
    # the 3 cols in nice number format
    salary_df['Min_Salary'] = salary_df['Min_Salary_Num'].apply(lambda x: '${:,.0f}'.format(x))
    salary_df['Max_Salary'] = salary_df['Max_Salary_Num'].apply(lambda x: '${:,.0f}'.format(x))
    salary_df['Avg_Salary'] = salary_df['Avg_Salary_Num'].apply(lambda x: '${:,.0f}'.format(x))
    # remove our intermediary columns
    return salary_df[['Min_Salary','Max_Salary','Avg_Salary','Min_Salary_Num','Max_Salary_Num','Avg_Salary_Num']]

In [31]:
salary_df = fix_salary(grab_salary(jobs_df,
                                   salary_patterns= [
                                       '\$\d{2,3},{0,1}\d{3}',
                                       '\$\d{2,3} {0,1}k'
                                   ]))
salary_df

,Min_Salary,Max_Salary,Avg_Salary,Min_Salary_Num,Max_Salary_Num,Avg_Salary_Num
53,"$119,442","$119,442","$119,442",119442.0,119442.0,119442.0
180,"$250,000","$250,000","$250,000",250000.0,250000.0,250000.0
233,"$100,000","$100,000","$100,000",100000.0,100000.0,100000.0
242,"$100,000","$140,000","$120,000",100000.0,140000.0,120000.0
260,"$111,077","$111,077","$111,077",111077.0,111077.0,111077.0
292,"$130,000","$130,000","$130,000",130000.0,130000.0,130000.0
394,"$175,000","$175,000","$175,000",175000.0,175000.0,175000.0
456,"$111,077","$111,077","$111,077",111077.0,111077.0,111077.0
475,"$60,000","$80,000","$70,000",60000.0,80000.0,70000.0
500,"$120,000","$120,000","$120,000",120000.0,120000.0,120000.0


In [32]:
# filter_state = jobs_df['Job Location'].str.contains('united',case=False)
# jobs_df['Job Location'][filter_state]

In [33]:
jobs_df1 = pd.concat([jobs_df,salary_df],axis='columns')
# make a copy of this for our raw data
jobs_df_withsal = jobs_df1.copy()
# split the state and city for tableau
# jobs_df_withsal['City'] = jobs_df_withsal['Job Location'].str.extract('^(.+),')
# jobs_df_withsal['State'] = jobs_df_withsal['Job Location'].str.extract(',(.+)$')
# remove all the NaNs
jobs_df_withsal.fillna('',inplace=True)
# make the state clean when the job puts the location in format State,Country
# filter_state = jobs_df_withsal['State'].str.contains('united',case=False)
# jobs_df_withsal['State'].loc[filter_state] = jobs_df_withsal['City'].loc[filter_state].apply(lambda x: us.states.lookup(x).abbr)
today = datetime.today().strftime('%Y-%M-%d')
jobs_df_withsal.to_csv(f'{keywords}_{location}_{today}.csv')
jobs_df_withsal

,Job Title,Company,Job Location,Job URL,Job Description,Seniority Level,Employment Type,Job Function,Industries,Min_Salary,Max_Salary,Avg_Salary,Min_Salary_Num,Max_Salary_Num,Avg_Salary_Num
0,Associate Solution Engineer - Sydney,Appian Corporation,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2622582377/,"At Appian, we’re seeking to grow our culture ...",Associate,Full-time,Engineering and Information Technology,Computer Software,,,,,,
1,Presales solution Engineer,Powell Software,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2649187443/,,Associate,Full-time,Sales and Information Technology,Information Technology and Services,,,,,,
2,Service Cloud Solution Engineer,Salesforce,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/2652778606/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ...",,,,,,
3,Associate Solution Engineer,Salesforce,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2661212338/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ...",,,,,,
4,Solution Engineer,Salesforce,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2661268390/,"To get the best candidate experience, please ...",Not Applicable,Full-time,Engineering and Information Technology,"Computer Software, Information Technology and ...",,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,RAMS Engineer,Jobs on jobactive,"Docklands, Victoria, Australia",https://www.linkedin.com/jobs/view/2659088142/,We currently have a career defining opportuni...,Entry level,Full-time,Engineering and Information Technology,"Restaurants, Food & Beverages, and Design",,,,,,
562,Field Services Engineer - Mechanical (Turbomac...,Baker Hughes,"Moomba, South Australia, Australia",https://www.linkedin.com/jobs/view/2588177238/,"This is not an active position, this posting ...",Entry level,Full-time,Information Technology,"Information Technology and Services, Computer ...",,,,,,
563,Mechanical Engineer,Wave International Pty Ltd,"Brisbane, Queensland, Australia",https://www.linkedin.com/jobs/view/2650782889/,Leading Engineering Consultancy working from ...,Entry level,Full-time,Engineering and Information Technology,"Staffing and Recruiting, Oil & Energy, and Man...",,,,,,
564,Dynamics 365 F&O Solution Architect,Microsoft,"Melbourne, Victoria, Australia",https://www.linkedin.com/jobs/view/2486220651/,The Microsoft Asia Business Application Domai...,Not Applicable,Full-time,Engineering and Information Technology,"Computer Hardware, Computer Software, and Info...",,,,,,


In [48]:
# for the reddit post
# use this for markdown tables https://www.tablesgenerator.com/markdown_tables
jobs_df1['Salary_num'] = pd.to_numeric(jobs_df1['Salary'].str.replace('\$|,|k','')) # make it a number for sorting
jobs_df1 = jobs_df1.sort_values(by='Salary_num',ascending=False)
filter_sal = jobs_df1['Salary'].notnull()
filter_analyst = jobs_df1['Job Title'].str.contains('analyst',case=False)
# make the column order a bit nicer
col_order = ['Salary', 'Job Title',
 'Company',
 'Job Location',
 'Job URL'
 ]
jobs_sal_df = jobs_df1[filter_sal & filter_analyst][col_order]
jobs_sal_df.to_csv('test.csv',index=False)

In [34]:
jobs_df1 = jobs_df1.sort_values(by='Min_Salary_Num',ascending=False)
filter_sal = jobs_df1['Min_Salary_Num'].notnull()
# make the column order a bit nicer
col_order = ['Min_Salary','Max_Salary','Avg_Salary','Job Title','Company','Job Location',]
jobs_sal_df = jobs_df1[filter_sal][col_order]
jobs_sal_df

,Min_Salary,Max_Salary,Avg_Salary,Job Title,Company,Job Location
180,"$250,000","$250,000","$250,000",Development Manager/Solutions Architect,FIRESOFT People,"Sydney, New South Wales, Australia"
394,"$175,000","$175,000","$175,000",Solution Architect - Digital & Mobile App,Jobx Pty Ltd,"Sydney, New South Wales, Australia"
292,"$130,000","$130,000","$130,000",Solution Architect - Data & Cloud Delivery,Command Recruitment Group,"Sydney, New South Wales, Australia"
500,"$120,000","$120,000","$120,000",Lead Platform Engineer - Enterprise Hcm | Peop...,Randstad Education Australia,"Brisbane, Queensland, Australia"
510,"$120,000","$140,000","$130,000",Lead Platform Engineer - Enterprise Hcm | Peop...,Randstad Education Australia,"Brisbane, Queensland, Australia"
53,"$119,442","$119,442","$119,442",Solution architect,City of Boroondara,"Camberwell, Victoria, Australia"
260,"$111,077","$111,077","$111,077",Senior Software Engineer - Multiple Roles,NSW Department of Customer Service,"Sydney, New South Wales, Australia"
456,"$111,077","$111,077","$111,077",Senior Software Engineer - Multiple Roles,I work for NSW,"Sydney, New South Wales, Australia"
233,"$100,000","$100,000","$100,000",Senior Solution Consultant - Hybrid Presales a...,DXC Technology,"Macquarie Park, New South Wales, Australia"
242,"$100,000","$140,000","$120,000",Data Engineer / Sr. Data Analyst,Troocoo,"Melbourne, Victoria, Australia"


## Finding which tech stack is most in demand

In [35]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
import nltk

In [36]:
# nltk doesn't come with everything
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jason.chiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# stop is a list of stopwords
stop = stopwords.words('english')
newStopWords = []
stop.extend(newStopWords)

In [39]:
# multi word tech stack
multi_tech = {
    'power bi':'power_bi',
    'powerbi':'power_bi',
    'nosql':'no_sql',
    'no sql':'no_sql',
    'data studio':'data_studio',
    'datastudio':'data_studio',
    ' r ':' _r_ ',
    'datarobot':'data_robot',
    'data robot':'data_robot',
    'google cloud platform':'gcp'
}
darlo_multi ={
    'adobe xd':'adobe_xd',
    'web design':'web_design',
    'webdesign':'web_design',
    'mobile app':'mobile_app',
    'mobile application':'mobile_app',
    'product design':'product_design',
    'user interface':'user_interface',
    'adobe creative cloud':'adobe_creative_cloud',
    'adobe suite':'adobe_creative_cloud',
    'optimal sort':'optimal_sort',
    'photoshop':'adobe_photoshop',
    'illustrator':'adobe_illustrator',
    'after effect':'adobe_after_effects'
}

In [18]:
jobs_df = pd.read_csv('ux designer_Australia_2021-59-31.csv')
jobs_df.head()

,Unnamed: 0,Job Title,Company,Job Location,Job URL,Job Description,Seniority Level,Employment Type,Job Function,Industries,Min_Salary,Max_Salary,Avg_Salary,Min_Salary_Num,Max_Salary_Num,Avg_Salary_Num
0,0,Lead UX/UI Designer,Robert Walters,"Perth, Western Australia, Australia",https://www.linkedin.com/jobs/view/2645697068/,This opportunity is within a leading geoscien...,Mid-Senior level,Full-time,Information Technology,Mining & Metals,NaN,NaN,NaN,NaN,NaN,NaN
1,1,UX Designer / Researcher,Endeavour Group,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2496656500/,About UsEndeavour Group was created in 2020 t...,Mid-Senior level,Full-time,"Design, Art/Creative, and Information Technology","Computer Software, Retail, and Financial Services",NaN,NaN,NaN,NaN,NaN,NaN
2,2,UX Designer,Endeavour Group,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2642705335/,About UsEndeavour Group was created in 2020 t...,Mid-Senior level,Full-time,"Design, Art/Creative, and Information Technology","Computer Software, Retail, and Financial Services",NaN,NaN,NaN,NaN,NaN,NaN
3,3,Senior UX Designer,Peoplebank,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2619882773/,"Location: Sydney, New South Wales Job Type: P...",Associate,Contract,"Design, Art/Creative, and Information Technology","Information Technology and Services, Computer ...",NaN,NaN,NaN,NaN,NaN,NaN
4,4,Senior UX Researcher / Service Designer,Objective Experience,"Sydney, New South Wales, Australia",https://www.linkedin.com/jobs/view/2661207413/,We are looking for a brilliant researcher / s...,NaN,Full-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
words = jobs_df['Job Description'].to_frame()
words.columns = ['words']
# below regex removes any non-word, non-space characters. This will mess up some words, but we're only interested in tech we know about, so it doesn't matter much
words['words'] = words['words'].str.replace("[^\w\s]", "").str.lower()
words['words'] = words['words'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
for key in multi_tech:
    words['words'] = words['words'].str.replace(key,multi_tech[key])
# for key in darlo_multi:
#     words['words'] = words['words'].str.replace(key,darlo_multi[key])

freq = pd.DataFrame(words['words'].str.split(expand=True).stack().value_counts()).reset_index()
freq.columns = ['words','count']
freq.head(30)

,words,count
0,experience,2888
1,technical,2273
2,solutions,2068
3,business,1883
4,work,1778
5,team,1593
6,solution,1495
7,customer,1477
8,customers,1459
9,skills,1259


In [25]:
# for love Adobe suite, Lookback, Axure, Sketch, InVision, Omnigraffle, Userzoom, Optimal sort, Figma
# to get phrases, replace powerbi and power bi with power_bi
tech1 = ['figma','sketch','invision','lookback',
        'askable','miro','confluence','jira','axure',
        'omnigraffle','userzoom','html','css','javascript']
tech2 = tech1 + (list(set(darlo_multi.values())))
tech = '|'.join(tech2)
# these words contain things like aws and excel
exclude1 = ['capital','laws','excellen']
exclude = '|'.join(exclude1)

filter3 = freq['words'].str.contains(tech)
filter4 = freq['words'].str.contains(exclude)

# freq[(filter3) & ~(filter4)].head(10)
# freq[filter5]

top10_tech_df = freq[(filter3) & ~(filter4)].head(10)
# make the names nicer
top10_tech_df['words'] = top10_tech_df['words'].str.replace('_',' ').str.strip().str.upper()
top10_tech_df

,words,count
135,PRODUCT DESIGN,104
143,PRODUCT DESIGNER,100
146,FIGMA,97
186,SKETCH,81
317,INVISION,51
412,PRODUCT DESIGNERS,41
418,USER INTERFACE,40
714,CSS,23
730,JIRA,23
773,HTML,22


In [41]:
# freq is a word (single word) count
# to get phrases, replace powerbi and power bi with power_bi
tech1 = ['sql','python','excel',
        'tableau','aws','gcp',
        'azure','qlik','matlab','hadoop',
        'looker','domo','dbt', 'kafka','spark'
        'datorama','_r_']
tech2 = tech1 + (list(set(multi_tech.values())))
tech = '|'.join(tech2)
# these words contain things like aws and excel
exclude1 = ['capital','laws','excellen']
exclude = '|'.join(exclude1)

filter3 = freq['words'].str.contains(tech)
filter4 = freq['words'].str.contains(exclude)

# freq[(filter3) & ~(filter4)].head(10)
# freq[filter5]

top10_tech_df = freq[(filter3) & ~(filter4)].head(10)
# make the names nicer
top10_tech_df['words'] = top10_tech_df['words'].str.replace('_',' ').str.strip().str.upper()
top10_tech_df

,words,count
49,AZURE,539
88,AWS,348
429,SQL,102
538,PYTHON,85
920,TABLEAU,46
1394,GCP,27
1571,EXCEL,23
1631,HADOOP,22
1720,NO SQL,20
1910,POWER BI,17


# Writing the data to our database

In [267]:
# looks like we need none of the above, the auth is contained inside pygsheets
# need to enable google sheets api and google drive api 
# then share your spreadsheet with the client_email in your key.json file
import pygsheets
client = pygsheets.authorize(service_file='jason2021-key.json')

In [268]:
# trying one from jason@wvc, also one from jasonsan to share
# so it doesn't matter who the owner of the google sheet is, only that we share it with the client_email
client.spreadsheet_titles()

['Pvolo - Bag1', 'Data Analyst Course planning', 'Linkedin Learning Project']

In [269]:
# open spreadsheet name
spreadsheet1 = client.open("Linkedin Learning Project")


In [270]:
# check the names of the tabs or worksheets
spreadsheet1.worksheets()

[<Worksheet 'Raw Data' index:0>,
 <Worksheet 'Top 10 Skills' index:1>,
 <Worksheet 'Salaries' index:2>]

In [271]:
# open the worksheet or tab
worksheet1 = spreadsheet1.worksheet_by_title('Raw Data')

In [272]:
# Put sheet into a dataframe
df = worksheet1.get_as_df()
df.head()

,Job Title,Company,Job Location,Job URL,Job Description,Seniority Level,Employment Type,Job Function,Industries,Min_Salary,Max_Salary,Avg_Salary,Min_Salary_Num,Max_Salary_Num,Avg_Salary_Num,City,State
0,"Data Analyst, Research and Insights",Google,"New York, NY",https://www.linkedin.com/jobs/view/2616405818/...,Note: By applying to this position you will h...,Not Applicable,Full-time,Information Technology,Information Services and Internet,,,,,,,New York,NY
1,Tableau Business Intelligence Senior Analyst,Salesforce,"San Francisco, CA",https://www.linkedin.com/jobs/view/2615511362/...,"To get the best candidate experience, please ...",Not Applicable,Full-time,"Research, Analyst, and Information Technology","Computer Software, Information Technology and ...",,,,,,,San Francisco,CA
2,"Data Analyst, Churn Analytics",Disney Streaming Services,"New York, NY",https://www.linkedin.com/jobs/view/2620559153/...,Job Summary: We are looking for an experience...,Mid-Senior level,Full-time,"Quality Assurance, Information Technology, and...","Marketing and Advertising, Computer Software, ...",,,,,,,New York,NY
3,Product Data Analyst II,Uber,"New York, NY",https://www.linkedin.com/jobs/view/2632664783/...,About The RoleAre you interested in driving b...,Not Applicable,Full-time,Information Technology,"Computer Software, Consumer Services, and Info...",,,,,,,New York,NY
4,Senior Data Analyst - Content,Hulu,"Santa Monica, CA",https://www.linkedin.com/jobs/view/2620557390/...,SummaryThe Disney Median & Entertainment Dist...,Mid-Senior level,Full-time,"Business Development, Analyst, and Engineering","Marketing and Advertising, Computer Software, ...",,,,,,,Santa Monica,CA


In [220]:
# in the tab write our dataframe with a starting coordinates row,col
worksheet1.set_dataframe(jobs_df_withsal,start=(1,1))

In [ ]:
# next the worksheet or tab
worksheet2 = spreadsheet1.worksheet_by_title('Top 10 Skills')
worksheet2.set_dataframe(top10_tech_df,start=(1,1))

In [223]:
# next the worksheet or tab
worksheet3 = spreadsheet1.worksheet_by_title('Salaries')
worksheet3.set_dataframe(jobs_sal_df,start=(1,1))